In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime
import chromedriver_binary
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#!pip install chromedriver-binary==87.0.4280.88

# chromedriverのバージョン自動更新
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get('https://google.com')

You should consider upgrading via the '/Users/ruka/.pyenv/versions/3.8.3/bin/python3.8 -m pip install --upgrade pip' command.


In [3]:
year = [2018,2019,2020]
#year = list(range(1983,2021)) # range
print(year)

[2018, 2019, 2020]


In [4]:
def get_dfs(stock_number):
    dfs = []
    for y in year:
        try:
            #print(y)
            url = 'https://kabuoji3.com/stock/{}/{}/'.format(stock_number,y)
            headers = {
                "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
            }
            soup = BeautifulSoup(requests.get(url, headers = headers).content,'html.parser')
            tag_tr = soup.find_all('tr')
            head = [h.text for h in tag_tr[0].find_all('th')]
            data = []
            for i in range(1,len(tag_tr)):
                data.append([d.text for d in tag_tr[i].find_all('td')])
            df = pd.DataFrame(data, columns = head)

            col = ['始値','高値','安値','終値','出来高','終値調整']
            for c in col:
                df[c] = df[c].astype(float)
            df['日付'] = [datetime.strptime(i,'%Y-%m-%d') for i in df['日付']]
            dfs.append(df)
        except IndexError:
            pass
            #print('No data')
    return dfs

def concatenate(dfs):
    data = pd.concat(dfs,axis=0)
    data = data.reset_index(drop=True)
    col = ['始値','高値','安値','終値','出来高','終値調整']
    for c in col:
        data[c] = data[c].astype(float)
    return data

In [5]:
# 全体を更新する場合
code_list = pd.read_csv('tokyo1-2.csv') # 東証１部と２部全体

# 一部を更新する場合
#code_list = pd.read_csv('id_test.csv')

print(len(code_list))
code_list[:10]

2657


,code,name
0,1301,極洋
1,1332,日本水産
2,1333,マルハニチロ
3,1352,ホウスイ
4,1375,雪国まいたけ
5,1376,カネコ種苗
6,1377,サカタのタネ
7,1379,ホクト
8,1384,ホクリヨウ
9,1413,ヒノキヤグループ


In [6]:
%%time
driver = webdriver.Chrome()
error_list = []

for i in range(len(code_list)):
    k = code_list.loc[i,'code']
    v = code_list.loc[i,'name']
    #print(k,v)
    try:
        dfs = get_dfs(k)
        data = concatenate(dfs) 
        data.to_csv('data/{}-{}.csv'.format(k,v))
    except:
        error_list.append(k)

CPU times: user 24min 17s, sys: 29.2 s, total: 24min 46s
Wall time: 1h 46min 10s


In [ ]:
error_list.to_csv('error_list.csv')

In [1]:
#end